vue笔记

`vue`使用`defineProperty`去监听绑定到对象的属性。
在`get` `set` 方法中，不能直接使用`this`去返回`a`的属性值，会造成循环调用

In [6]:
{
    let a = {};
    let tempa = {str:'-'};
    Object.defineProperty(a,'str',{
        get:function(){
            console.log('you trigger get:',tempa.str);
            return tempa.str;
        },
        set:function(newStr){
            console.log('you trigger set:',newStr);
            tempa.str = newStr;
        }
    })
    console.log(a.str);
    a.str='1';
}

you trigger get: -
-
you trigger set: 1


'1'

对数组的监听是通过修改数组的方法完成的,然后通过原型上`Array`的方法实现真正的方法调用。

In [9]:
{
    let arr = [];
    let proto = Array.prototype,
        slice = proto.slice,
        mutatorMethods = ['pop','push'];
   mutatorMethods.forEach(method=>{
       arr[method]=function(...argv){
           console.log('argv:',[...argv]);
           proto[method].apply(this,argv);
           console.log({
               event:method,
               args:slice.call(argv),
               array:arr
           })
       }
   })
   arr.push(1,2,3)
   arr.pop()
   console.log('arr',arr);
}

argv: [ 1, 2, 3 ]
{ event: 'push',
  args: [ 1, 2, 3 ],
  array: [ 1, 2, 3, pop: [Function], push: [Function] ] }
argv: []
{ event: 'pop',
  args: [],
  array: [ 1, 2, pop: [Function], push: [Function] ] }
arr [ 1, 2, pop: [Function], push: [Function] ]


用`proxy`代替`defineProperty`

In [6]:
{
    let a = {};
    let tempa = Object.create({});
    let handler = {
        get:function(target,name){
            console.log(`set:-target->${target},name->${name}`);
            console.log(`target[${name}]:${target[name]}`);
            //使用Reflct
            Reflect.get(target,name);
        },
        set:function(target,name,value){
            console.log(`set:-target->${target},name->${name},value->${value}`);
            //target[name]=value;
            Reflect.set(target,name,value)
            console.log(`target[${name}]:${target[name]}`);
        }
    }
    
    let reala = new Proxy(a,handler);
    reala.x=1;
    reala.y;
    reala.x;
}

set:-target->[object Object],name->x,value->1
target[x]:1
set:-target->[object Object],name->y
target[y]:undefined
set:-target->[object Object],name->x
target[x]:1


reflect and proxy have completely different purposes and different capabilities.

the `Proxy` object is used to define custom behavior for fundamental operations (e.g. property lookup, assignment, enumeration, function invocation,etc).

`Reflect` is a built-in object that provides methods for interceptable javascript operations. the methods are the same as those of proxy handlers.

In [3]:
{
    const obj = {
        a:'foo',
        b:'bar'
    };
    
    console.log('obj.a:',obj.a);
    console.log('Reflct:',Reflect.get(obj,'a'));
    console.log('obj.c:',obj.c);
    console.log('Reflct:',Reflect.get(obj,'c'));
    
    const proxy = new Proxy(obj,{
        get(target,property){
            return property in target? target[property]:'default';
        }
    })
    
    console.log('obj.a:',proxy.a);
    console.log('Reflct:',Reflect.get(proxy,'a'));
    console.log('obj.c:',proxy.c);
    console.log('Reflct:',Reflect.get(proxy,'c'));
}

obj.a: foo
Reflct: foo
obj.c: undefined
Reflct: undefined
obj.a: foo
Reflct: foo
obj.c: default
Reflct: default


```javascript
    new Proxy(obj,{
        get:Reflect.get
    })
```